In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import import_ipynb
from data_loader import NTUDataloader
from va_cnn_model import VA

importing Jupyter notebook from data_loader.ipynb
importing Jupyter notebook from va_cnn_model.ipynb


In [2]:
def main():
    
    model = VA(60)
    model = model.cuda()
    
    # We train each stream of view adaptive neural network by minimizing the cross-entropy loss end-to-end. 
    criterion = nn.CrossEntropyLoss().cuda()
    
    # Adam is adapted to train all networks, and the initial learning rate is set to 0.0001 for all datasets
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    
    loader = NTUDataloader()
    train_loader = loader.train_data_loader()
    val_loader = loader.val_data_loader()
    # print(train_loader)
    max_epochs = 10
    
    for epoch in range(max_epochs):
        
        # running_loss = 0
#         tot = 0
#         count = 0
        # get the inputs data is a list of [inputs, maxmin, labels]
        model.train()
        for i,data in enumerate(train_loader):
            inputs, maxmin, outlabels = data
            
            # forward + backward + optimize
            outputs, image, params = model(inputs.cuda(), maxmin.cuda())
            outlabels = outlabels.cuda()
            loss = criterion(outputs, outlabels)

            # zero the parameter gradients
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
#             count += inputs.size(0)
#             batch_size = outlabels.size(0)
#             _, pred = outputs.topk(1, 1, True, True)
#             pred = pred.t()
#             correct = pred.eq(outlabels.view(1, -1).expand_as(pred))
#             correct = correct.view(-1).float().sum(0, keepdim=True)
#             acc =  correct.mul_(100.0 / batch_size)
#             tot = acc*batch_size
# #             print(acc.cpu().numpy())
#             print('Epoch-{:<3d}' 'Accuracy {accuracy:.4f}'.format(epoch + 1, accuracy=acc.cpu().numpy()[0]))  

            # calculating loss and accuracy
            # running_loss += loss.item()
        
            if (i+1) % 200 == 0:    # print every 10 mini-batches
                print('Epoch-{:<3d} {:3d} batches\t'
                      'loss {loss:.4f}'.format(
                       epoch + 1, i + 1,loss=loss))
            
        validate(val_loader, model, criterion, epoch)

    print('Finished Training')
    print('')
    print('Now Testing')
    test_loader = loader.test_data_loader()
    test(test_loader, model, criterion)
    
    
def validate(val_loader, model, criterion, epoch):
    model.eval()
    tot = 0
    count = 0
    for i,data in enumerate(val_loader):
        inputs, maxmin, outlabels = data
            
        # forward + backward + optimize
        with torch.no_grad():
            outputs, image, params = model(inputs.cuda(), maxmin.cuda())
        outlabels = outlabels.cuda()
        with torch.no_grad():
            loss = criterion(outputs, outlabels)
        count += inputs.size(0)
        batch_size = outlabels.size(0)
        _, pred = outputs.topk(1, 1, True, True)
        pred = pred.t()
        correct = pred.eq(outlabels.view(1, -1).expand_as(pred))
        correct = correct.view(-1).float().sum(0, keepdim=True)
        acc =  correct.mul_(100.0 / batch_size)
        tot += acc.cpu().numpy()[0]*batch_size
    print('')
    print('Validation: Epoch-{:<3d}' 'Accuracy {accuracy:.4f}'.format(epoch + 1, accuracy=tot/count))        
    print('')
    
def test(test_loader, model, criterion):
    model.eval()
    tot = 0
    count = 0
    for i,data in enumerate(test_loader):
        inputs, maxmin, outlabels = data
            
        # forward + backward + optimize
        with torch.no_grad():
            outputs, image, params = model(inputs.cuda(), maxmin.cuda())
        outlabels = outlabels.cuda()
        with torch.no_grad():
            loss = criterion(outputs, outlabels)
        count += inputs.size(0)
        batch_size = outlabels.size(0)
        _, pred = outputs.topk(1, 1, True, True)
        pred = pred.t()
        correct = pred.eq(outlabels.view(1, -1).expand_as(pred))
        correct = correct.view(-1).float().sum(0, keepdim=True)
        acc =  correct.mul_(100.0 / batch_size)
        tot += acc.cpu().numpy()[0]*batch_size
    print('')
    print('Testing: Total-{:<3d}' '  Accuracy {accuracy:.4f}'.format(count, accuracy=tot/count))        
    print('')
        


In [3]:
main()

(38086, 300, 150)
(16487, 300, 150)
Epoch-1   200 batches	loss 2.1145
Epoch-1   400 batches	loss 1.3451
Epoch-1   600 batches	loss 1.3268
Epoch-1   800 batches	loss 1.0231
Epoch-1   1000 batches	loss 0.7354

Validation: Epoch-1  Accuracy 69.8753

Epoch-2   200 batches	loss 0.8353
Epoch-2   400 batches	loss 0.3947
Epoch-2   600 batches	loss 0.5571
Epoch-2   800 batches	loss 0.5922
Epoch-2   1000 batches	loss 0.7208

Validation: Epoch-2  Accuracy 71.8204

Epoch-3   200 batches	loss 1.2502
Epoch-3   400 batches	loss 0.4216
Epoch-3   600 batches	loss 0.4066
Epoch-3   800 batches	loss 0.8972
Epoch-3   1000 batches	loss 0.4500

Validation: Epoch-3  Accuracy 72.8678

Epoch-4   200 batches	loss 0.3949
Epoch-4   400 batches	loss 0.2627
Epoch-4   600 batches	loss 0.5583
Epoch-4   800 batches	loss 0.4016
Epoch-4   1000 batches	loss 0.5577

Validation: Epoch-4  Accuracy 78.6035

Epoch-5   200 batches	loss 0.4768
Epoch-5   400 batches	loss 0.2661
Epoch-5   600 batches	loss 0.1614
Epoch-5   800 batc